<a href="https://colab.research.google.com/github/Pmilivojevic/Bioinformatics/blob/main/bioinformatics_chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import copy
from google.colab import drive
drive.mount('/content/drive/')
nucleotides= ['A', 'C', 'G', 'T']
from itertools import product

Mounted at /content/drive/


In [ ]:
def composition(seq, k):
  kmers= []

  for i in range(len(seq)-k+1):
    kmers.append(seq[i:i+k])

  return kmers

def genomePath(kmers):
  seq= kmers[0]

  for i in range(1, len(kmers)):
    seq+= kmers[i][len(kmers[0])-1]

  return seq

def adjacencyList(kmers):
  #kmers.sort()
  adjList= []
  for kmer in kmers:
    l= []
    for i in range(len(kmers)):
      if kmer == kmers[i]:
        pass
      elif kmer[1:] == kmers[i][:len(kmer)-1]:
        if not l:
          l.append(kmer)
          l.append(kmers[i])
        else:
          l.append(kmers[i])
    if l:
      adjList.append(l)

  return adjList

def generate_binary_kmers(k):
    binList= []
    for i in range(2 ** k):
        res = bin(i).lstrip('0b')
        binList.append(res)
        yield '0' * (k - len(res)) + res

def check_universal(test_string, k):
    for kmer in generate_binary_kmers(k):
        if kmer not in test_string:
            return False
    return True

def find_universal_string(k):
    optimal_string_length = (2 ** k) + (k - 1)
    universal= []
    for test_string in generate_binary_kmers(optimal_string_length):
        if check_universal(test_string, k):
            universal.append(test_string)

    return universal

def DeBruijn(seq, k):
  nodes= []

  for i in range(len(seq)-k+2):
    if not nodes:
      nodes.append(seq[i:i+k-1])
    else:
      adding= True
      for node in nodes:
        if seq[i:i+k-1] == node:
          adding= False
          break
      if adding:
        nodes.append(seq[i:i+k-1])

  nodes.sort()
  kmers= composition(seq, k)
  kmers.sort()

  adjList= {}

  for node in nodes:
    tempList= []
    for  kmer in kmers:
      if kmer[:k-1] == node:
        tempList.append(kmer[1:])

    adjList[node]= tempList

  return adjList

def DeBruijnOfKmers(kmers):
  kmers.sort()

  adjList= {}

  for kmer in kmers:
    if kmer[:-1] not in adjList:
      adjList[kmer[:-1]]= []
    adjList[kmer[:-1]].append(kmer[1:])

  return adjList

def isBalanced(node, edges):
  indgN= 0
  outdgN= 0
  for edge in edges:
    if edge[1:] == node:
      indgN+= 1
    else:
      outdgN+= 1

  if indgN == outdgN:
    return True
  else:
    return False

def isEmpty(graph):
  for node in list(graph.values()):
    if len(node) != 0:
      return False
  return True

def isStartNode(graph, node):
  outDgr= len(graph[node])
  inDgr= 0

  for k in graph.keys():
    for v in graph[k]:
      if v == node:
        inDgr+= 1

  if outDgr > inDgr:
    return True
  else:
    return False

def first_element(graph: dict) -> str:
   temp = {k: len(graph[k]) for k in graph.keys()}
   for k,v in graph.items():
      for val in v:
         if val in temp.keys():
            temp[val]-=1
   maximum = list(temp.keys())[list(temp.values()).index(1)]
   return maximum

def endNode(graph):
  end= True
  ends= []
  for k in graph.keys():
    for v in graph[k]:
      for l in graph.keys():
        if v == l:
          end= False
          break
        else:
          end= True

      if end:
        #return v
        ends.append(v)
  return ends

def depthPath(graph, node, end, path):
  if end != node:
    if len(graph[node]) > 0:
      newNode= graph[node][0]
      path.append(newNode)
      graph[node].remove(newNode)
      depth(graph, newNode, end, path)

def depth(graph, node, path):
  if len(graph[node]) > 0:
    newNode= graph[node][0]
    path.append(newNode)
    graph[node].remove(newNode)
    depth(graph, newNode, path)

def EulerianCircuit(graph):
  i= 0
  tempGraph= dict(graph)

  while not isEmpty(tempGraph) and i < len(list(tempGraph.keys())):
    tempGraph= dict(graph)
    solution= []
    startNode= list(tempGraph.keys())[i]
    tempSolution= [startNode]
    depth(tempGraph, startNode, tempSolution)
    while len(tempSolution) > 1:
      solution.append(tempSolution[-1])
      tempSolution.pop()
      newNode= tempSolution[-1]
      depth(graph, newNode, tempSolution)
    solution.append(tempSolution[-1])
    i+= 1

  solution.reverse()
  if i == len(list(tempGraph.keys()))-1:
    if isEmpty(tempGraph):
      return solution
    else:
      print('Nema ojlerovog ciklusa')
  else:
    return solution

def EulerianPath(graph):
  end= endNode(graph)
  for k in graph.keys():
    if isStartNode(graph, k):
      tempGraph= dict(graph)
      solution= []
      startNode= k
      tempSolution= [startNode]
      depthPath(tempGraph, startNode, end, tempSolution)
      while len(tempSolution) > 1:
        solution.append(tempSolution[-1])
        tempSolution.pop()
        newNode= tempSolution[-1]
        depthPath(graph, newNode, end, tempSolution)
      solution.append(tempSolution[-1])

      if isEmpty(tempGraph):
        solution.reverse()
        return solution

  print('Nema ojlerovog ciklusa')

def euler_path(graph: dict) -> list:
  path = []
  n = first_element(graph)
 # print(n)
  stack = [n]
  while stack:
    u = stack[-1]
    if u not in graph.keys():
      path.append(stack.pop())
    elif graph[u]:
      w = graph[u][0]
      stack.append(w)
      del graph[u][0]
    else:
      path.append(stack.pop())
  path.reverse()
  return path

def StringReconstruction(patterns):
  db= DeBruijnOfKmers(patterns)
  kmers= euler_path(db)
  seq= genomePath(kmers)
  return seq

def StringSpelledByGappedPatterns(kmers, k, d):
  prefixKmers= []
  sufixKmers= []

  for i in range(len(kmers)-1):
    prefixKmers.append(kmers[i][:len(kmers[i])//2] + kmers[i+1][len(kmers[i+1])//2-1])
    sufixKmers.append(kmers[i][len(kmers[i])//2:] + kmers[i+1][len(kmers[i+1])-1])

  prefixSeq= genomePath(prefixKmers)
  sufixSeq= genomePath(sufixKmers)

  for i in range(k+d, len(prefixSeq)):
    if prefixSeq[i] != sufixSeq[i-k-d]:
      return 'There is no string spelled by the gapped patterns'

  return prefixSeq[:k+d] + sufixSeq

def outDegree(graph, node):
  return len(graph[node])

def inDegree(graph, node):
  inDgr= 0
  for k in graph.keys():
    for v in graph[k]:
      if v == node:
        inDgr+= 1

  return inDgr

def is_1_to_1(graph, node):
  if node in graph.keys():
    #print(node)
    if inDegree(graph, node) == 1 and outDegree(graph, node) == 1:
      return True
    else:
      return False
  else:
    return False

def is_memb(pths, node):
  for pth in pths:
    for n in pth:
      if node == n:
        return True
  return False

def MaximalNonBranchingPaths(graph):
  paths= []
  for node in graph.keys():
    if not is_1_to_1(graph, node):
      if outDegree(graph, node) > 0:
        for n in graph[node]:
          nonBranchingPath= []
          nonBranchingPath.append(node)
          nonBranchingPath.append(n)
          n_temp= n
          while is_1_to_1(graph, n_temp):
            if n_temp != n:
              nonBranchingPath.append(n_temp)
            n_temp= graph[n][0]
            if n_temp == '362':
              return nonBranchingPath
          if not n_temp in nonBranchingPath:
            nonBranchingPath.append(n_temp)
          paths.append(nonBranchingPath)

  for node in graph.keys():
    if is_1_to_1(graph, node):
      if not is_memb(paths, node):
        for n in graph[node]:
          cycle= []
          start= node
          flag= False
          cycle.append(node)
          cycle.append(n)
          n_temp= n
          while is_1_to_1(graph, n_temp):
            if n_temp != n:
              cycle.append(n_temp)
              if n_temp == start:
                flag= True
                break
            n_temp= graph[n][0]
          if flag:
            paths.append(cycle)

  return paths

In [ ]:
filePath= '/content/drive/MyDrive/Colab Notebooks/dataset_6207_2.txt'

graph= {}

with open(filePath, 'r') as file:
  for line in file:
    lines= line.split(':')
    v1= lines[1].replace('\n', '')
    v2= v1[1:]
    v3= v2.split(' ')
    graph[lines[0]]= []
    for v in v3:
      graph[lines[0]].append(v)

#print(inDegree(graph, '362'))
#print(outDegree(graph, '362'))
print(graph)
p= StringReconstruction(graph)
print(p)
#print(is_1_to_1(graph, '362'))

{'0': ['212'], '1': ['119'], '2': ['286'], '3': ['294'], '4': ['115'], '5': ['226'], '6': ['366'], '7': ['336'], '8': ['176'], '9': ['283'], '10': ['365'], '11': ['77'], '12': ['153'], '13': ['244'], '14': ['3'], '15': ['398'], '16': ['208'], '17': ['148'], '18': ['61'], '19': ['165'], '20': ['303'], '21': ['331'], '22': ['118'], '23': ['327'], '25': ['367'], '26': ['330'], '27': ['179'], '28': ['207'], '29': ['122'], '30': ['5'], '31': ['105'], '32': ['328'], '33': ['36'], '34': ['222'], '35': ['34'], '36': ['243', '396'], '37': ['73'], '38': ['250'], '39': ['320', '308'], '40': ['202'], '41': ['117'], '42': ['289'], '43': ['60'], '44': ['288'], '45': ['46'], '46': ['390'], '47': ['181'], '48': ['338'], '49': ['276'], '50': ['319'], '51': ['375'], '52': ['308'], '53': ['238'], '54': ['298'], '55': ['2'], '56': ['299'], '57': ['258'], '58': ['354'], '59': ['188'], '60': ['135'], '61': ['268'], '62': ['163'], '63': ['108'], '64': ['95'], '65': ['200'], '66': ['350'], '67': ['220'], '68'

AttributeError: ignored

In [ ]:
kmers= ['AAAT', 'AATG', 'ACCC', 'ACGC', 'ATAC', 'ATCA', 'ATGC', 'CAAA', 'CACC', 'CATA', 'CATC', 'CCAG', 'CCCA', 'CGCT', 'CTCA', 'GCAT', 'GCTC', 'TACG', 'TCAC', 'TCAT', 'TGCA']
#print(kmers[-1:0:-1])
graph= {'1': ['2'], '2': ['3'], '3': ['4', '5'], '6': ['7'], '7': ['6']}
graph1= {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': []}

#print(is_1_to_1(graph, '7'))
p= StringReconstruction(kmers)
print(p)
print(len(p))

CAAATGCATACGCTCATCACCCAG
24


In [ ]:
filePath= '/content/drive/MyDrive/Colab Notebooks/dataset_203_7.txt'
'''with open(filePath, 'r') as file:
    graph = dict((line.strip().split(': ') for line in file))
    for key in graph:
        graph[key] = graph[key].split(' ')'''
lines= []
with open(filePath, 'r') as file:
  for line in file:
    lines.append(line)
text= ''
patterns= []
for c in lines[1]:
  if c != ' ':
    text+= c
  else:
    patterns.append(text)
    text= ''

graph= DeBruijnOfKmers(patterns)
#end= endNode(graph)
#i= 0
#for k in graph.keys():
 # i+= 1
  #if k == 'AGGGGAGGCATCAGATAACCATAA':
   # print(i)
print(endNode(graph))
#print(isStartNode(graph))
#print(first_element(graph))
#print(len(graph.items()))
txt= StringReconstruction(patterns)
print(txt)

with open('/content/drive/MyDrive/Colab Notebooks/result.txt', 'w') as f:
  f.write(txt)